In [13]:
from Script.SQLConn import SQLConn
from pyvis.network import Network
from collections import Counter
from colour import Color
import math
from difflib import SequenceMatcher

In [14]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [2]:
print('Connecting to the Database...')
SQLConn.instance()

Connecting to the Database...


In [3]:
net = Network(notebook=True)
number_of_shades = 10
colors = list(Color("green").range_to(Color("red"),number_of_shades))

In [4]:
SQLConn.instance().cursor.execute("""
    SELECT * FROM genre 
    INNER JOIN genre_artist ON genre.genre_id = genre_artist.genre_id
    INNER JOIN artist ON genre_artist.artist_id = artist.artist_id
    """)
result = SQLConn.instance().cursor.fetchall()
colnames = [desc[0] for desc in SQLConn.instance().cursor.description]

artists_genres = {}
genres = []
for genre in result:
    artist_id = genre[colnames.index('artist_id')] 
    if artist_id not in artists_genres:
        artists_genres[artist_id] = []
    artists_genres[artist_id].append(genre[colnames.index('genre')] )
    genres.append(genre[colnames.index('genre')])
    
genres = Counter(genres)
genres_color = []
for genre in genres:
    color = colors[math.floor(genres[genre] * number_of_shades/150.0)].hex_l
    genres_color.append(color)
    #print(str(genres[genre]) + ': ' + str(math.floor(genres[genre] * number_of_shades/150.0)) + ' - ' + color)


In [15]:
for genre in genres:
    print(str(similar(genre, 'pop')) + ': ' + genre)

0.25: contemporary post-bop
0.16666666666666666: cool jazz
0.0: ecm-style jazz
0.0: jazz
0.0: jazz funk
0.14285714285714285: jazz fusion
0.3076923076923077: jazz piano
0.1111111111111111: folk brasileiro
0.36363636363636365: nova mpb
0.25: pop lgbtq+ brasileira
0.4: pop nacional
0.35294117647058826: portuguese pop
0.11764705882352941: brazilian rock
0.0: latin alternative
0.0: manguebeat
0.3333333333333333: mpb
0.2727272727272727: pop rock brasileiro
0.13333333333333333: rock mineiro
0.07407407407407407: rock nacional brasileiro
0.4: metropopolis
0.07692307692307693: modern alternative rock
0.14285714285714285: modern rock
0.08333333333333333: american folk revival
0.21052631578947367: appalachian folk
0.0: blues
0.2857142857142857: folk
0.18181818181818182: old-time
0.23529411764705882: piedmont blues
0.1: traditional blues
0.10526315789473684: traditional folk
0.07692307692307693: irish singer-songwriter
1.0: pop
0.2857142857142857: rock
0.0: beatlesque
0.10526315789473684: british i

In [5]:
genres[genre]

1

In [6]:
edges = {}
for artist in artists_genres:
    genre_list = artists_genres[artist]
    for i in range(0, len(genre_list)):
        for j in range(i + 1, len(genre_list)):
            name = genre_list[i] + '-' + genre_list[j]
            if name not in edges:
                edges[name] = 0
            edges[name] = edges[name] + 1

net.add_nodes(list(genres.keys()), color=genres_color, value=[element * 1 for element in list(genres.values())])

for artist in artists_genres:
    genre_list = artists_genres[artist]
    for i in range(0, len(genre_list)):
        for j in range(i + 1, len(genre_list)):
            name = genre_list[i] + '-' + genre_list[j]
            if edges[name] > 3:
                net.add_edge(genre_list[i], genre_list[j])

In [11]:
net.show_buttons(filter_=['physics'])
net.show('mygraph.html')

In [8]:
net.get_node('classical')

{'color': '#208e00',
 'value': 27,
 'id': 'classical',
 'label': 'classical',
 'shape': 'dot'}

In [9]:
dict(sorted(genres.items(), key=lambda item: item[1], reverse=True)) 

{'pop': 149,
 'modern rock': 99,
 'dance pop': 84,
 'electropop': 80,
 'indie poptimism': 76,
 'rock': 71,
 'edm': 70,
 'tropical house': 62,
 'post-teen pop': 61,
 'modern alternative rock': 59,
 'pop dance': 59,
 'indie pop': 52,
 'pop rock': 48,
 'electro house': 48,
 'jazz': 46,
 'alt z': 46,
 'nova mpb': 40,
 'cool jazz': 39,
 'stomp and holler': 39,
 'jazz fusion': 34,
 'brazilian rock': 32,
 'indie folk': 32,
 'vapor soul': 31,
 'electronica': 31,
 'mpb': 30,
 'blues rock': 30,
 'pop edm': 30,
 'future bass': 29,
 'indie rock': 27,
 'vocal jazz': 27,
 'classical': 27,
 'indie soul': 26,
 'metropopolis': 25,
 'classic rock': 25,
 'pop rap': 25,
 'bebop': 25,
 'vapor twitch': 24,
 'modern blues rock': 22,
 'electric blues': 22,
 'alternative dance': 22,
 'hard bop': 22,
 'lo-fi beats': 22,
 'indietronica': 22,
 'indie electropop': 21,
 'neo mellow': 21,
 'soul jazz': 21,
 'jazz funk': 20,
 'blues': 20,
 'adult standards': 20,
 'art pop': 20,
 'soul': 19,
 'pop nacional': 18,
 'ind

In [10]:
genres

Counter({'contemporary post-bop': 8,
         'cool jazz': 39,
         'ecm-style jazz': 5,
         'jazz': 46,
         'jazz funk': 20,
         'jazz fusion': 34,
         'jazz piano': 11,
         'folk brasileiro': 9,
         'nova mpb': 40,
         'pop lgbtq+ brasileira': 6,
         'pop nacional': 18,
         'portuguese pop': 2,
         'brazilian rock': 32,
         'latin alternative': 1,
         'manguebeat': 3,
         'mpb': 30,
         'pop rock brasileiro': 11,
         'rock mineiro': 2,
         'rock nacional brasileiro': 9,
         'metropopolis': 25,
         'modern alternative rock': 59,
         'modern rock': 99,
         'american folk revival': 1,
         'appalachian folk': 1,
         'blues': 20,
         'folk': 5,
         'old-time': 1,
         'piedmont blues': 2,
         'traditional blues': 15,
         'traditional folk': 1,
         'irish singer-songwriter': 3,
         'pop': 149,
         'rock': 71,
         'beatlesque': 2,
    